# Proyecto Final Lab Inteligencia (Imagenes): Método alternativo 

In [1]:
#Imports
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy.spatial import distance as dist
import argparse
import glob

## 7) Uso de un método alternativo para la extracción de características

Se utiliza el método HoG (Histograms of Gradients) para realizar la extracción de caractrísticas. Se reutilizan varias de las funciones utilizadas anteriormente.

### 7.1) Lectura de imagen (en RGB)

In [2]:
#Funcion que lear una imagen a RGB
def leerRGB (nombreImagen):
    #Lectura
    readImage = cv2.imread(nombreImagen)
    return readImage

In [7]:
test = leerRGB('100501.jpg')

In [8]:
cv2.imshow('image',test)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
#Dimensiones de cada una de las imagenes
test.shape

(2448, 3264, 3)

### 7.2) División de la imagen en razones 1:2 (6filasx3columas)

In [10]:
#Funcion que recorte la imagen en 3x3 celdas. Devuelve 3 submatrices.
def div9Celdas(imRGB):
    #division
    R = imRGB[:,:,0]
    G = imRGB[:,:,1]
    B = imRGB[:,:,2]
    #Dimensiones de la imagen
    alto,ancho,prof = imRGB.shape
    #Nuevas dimensiones
    nuevoAlto = int(alto/6)
    nuevoAncho = int(ancho/3)
    #Creacion de las tres matrices (inicialmente vacias).
    Rdiv = np.zeros([nuevoAlto,nuevoAncho,18])
    Gdiv = np.zeros([nuevoAlto,nuevoAncho,18])
    Bdiv = np.zeros([nuevoAlto,nuevoAncho,18])
    #Dividir imagen
    vecDivisionesAlto = [0,nuevoAlto,nuevoAlto,nuevoAlto*2,nuevoAlto*2,nuevoAlto*3,nuevoAlto*3,nuevoAlto*4,nuevoAlto*4,nuevoAlto*5,nuevoAlto*5,nuevoAlto*6]
    vecDivisionesAncho = [0,nuevoAncho,nuevoAncho,nuevoAncho*2,nuevoAncho*2,nuevoAncho*3]
    count = 0
    count1 = 0
    while count1 <= 11:
        count2 = 0
        while count2 <= 5:
            Rdiv[:,:,count] = H[vecDivisionesAlto[count1]:vecDivisionesAlto[count1+1],vecDivisionesAncho[count2]:vecDivisionesAncho[count2+1]]
            Gdiv[:,:,count] = S[vecDivisionesAlto[count1]:vecDivisionesAlto[count1+1],vecDivisionesAncho[count2]:vecDivisionesAncho[count2+1]]
            Vdiv[:,:,count] = V[vecDivisionesAlto[count1]:vecDivisionesAlto[count1+1],vecDivisionesAncho[count2]:vecDivisionesAncho[count2+1]]
            count += 1
            count2 += 2
        count1 +=2  
        
    return np.uint8(Hdiv),np.uint8(Sdiv),np.uint8(Vdiv)

### 7.3) Cálculo de los gradientes.